In [1]:
import os
import pandas as pd
import requests
import json
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from config import API_key

In [2]:
url = 'https://api.data.gov/ed/collegescorecard/v1/schools?'

In [3]:
# First we need to use pandas to begin creating the final database
# with pandas we will read in the relevant CSVs from Resources ('salary_by_degree.csv', 'salary_by_college.csv')
# note: adding a third CSV to get the IPED6 number for the institutions (used in the API requests)

degreepath = os.path.join('..', 'Resources', 'salary_by_degree.csv')
collegepath = os.path.join('..', 'Resources', 'salary_by_college.csv')
idpath = os.path.join('..', 'Resources', 'OPEID.csv')

degreeDF = pd.read_csv(degreepath)
collegeDF = pd.read_csv(collegepath)
idDF = pd.read_csv(idpath)

In [4]:
# get IPED6 values for colleges in collegeDF (remember that some are named differently)
# method: use an inner join to find which ones are named differently in collegeDF
#         then replace the names so that we can once again inner join and have the full collegeDF represented
#
#  NOTE: Did some string manipulation to change the names programatically
#
#  NOTE: I have no idea why the only intuitive way to do this is deprecated

for idx in collegeDF.index:
    name = collegeDF.get_value(idx, 'School Name')
    paridx = name.find('(')
    if paridx != -1:
        collegeDF.set_value(idx, 'School Name', name[:paridx - 1])
    else:
        collegeDF.set_value(idx, 'School Name', name)

testDF = collegeDF.join(idDF.set_index('Name'), on = 'School Name', how = 'inner')

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


In [5]:
# LARGELY A TEST CELL, ONLY TWO FUNCTIONAL LINES OF CODE HERE #
###############################################################
###############################################################

#testDF.shape
#testDF[testDF['IPED6'].isnull()]
updatedDF = testDF
#testDF.head(45)
#updatedDF.head()
#print(updatedDF['IPED6'].value_counts())
#############    updatedDF.index.value_counts()
#############    ^ this must be why .get_value() is deprecated

updatedDF = updatedDF[~updatedDF.index.duplicated(keep='first')]

In [6]:
# make some API calls based on the IPED6 of colleges in updatedDF
# 

schools = {}
querystring = 'school.name,latest.admissions.admission_rate.overall,latest.admissions.sat_scores.midpoint.math,latest.admissions\
.sat_scores.midpoint.critical_reading,latest.admissions.sat_scores.midpoint.writing,latest.admissions.act_scores.midpoint.cumulative,latest\
.admissions.act_scores.midpoint.english,latest.admissions.act_scores.midpoint.math,latest.admissions.act_scores.midpoint.writing,\
latest.admissions.sat_scores.average.overall,latest.academics.program_percentage.agriculture,latest.academics.program_percentage.\
resources,latest.academics.program_percentage.architecture,latest.academics.program_percentage.ethnic_cultural_gender,\
latest.academics.program_percentage.communication,latest.academics.program_percentage.communications_technology,\
latest.academics.program_percentage.computer,latest.academics.program_percentage.personal_culinary,latest.academics.program_percentage.education,\
latest.academics.program_percentage.engineering,latest.academics.program_percentage.engineering_technology,\
latest.academics.program_percentage.language,latest.academics.program_percentage.family_consumer_science,\
latest.academics.program_percentage.legal,latest.academics.program_percentage.english,latest.academics.program_percentage.humanities,\
latest.academics.program_percentage.library,latest.academics.program_percentage.biological,latest.academics.program_percentage.\
mathematics,latest.academics.program_percentage.military,latest.academics.program_percentage.multidiscipline,\
latest.academics.program_percentage.parks_recreation_fitness,latest.academics.program_percentage.philosophy_religious,\
latest.academics.program_percentage.theology_religious_vocation,latest.academics.program_percentage.physical_science,\
latest.academics.program_percentage.science_technology,latest.academics.program_percentage.psychology,\
latest.academics.program_percentage.security_law_enforcement,latest.academics.program_percentage.public_administration_social_service,\
latest.academics.program_percentage.social_science,latest.academics.program_percentage.construction,latest.academics.program_percentage.\
mechanic_repair_technology,latest.academics.program_percentage.precision_production,latest.academics.program_percentage.transportation,\
latest.academics.program_percentage.visual_performing,latest.academics.program_percentage.health,latest.academics.program_percentage.\
business_marketing,latest.academics.program_percentage.history,latest.student.size,latest.student.demographics.race_ethnicity.white,\
latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.hispanic,latest.student.demographics.race_ethnicity.\
asian,latest.student.demographics.race_ethnicity.aian,latest.student.demographics.race_ethnicity.nhpi,latest.student.demographics.race_ethnicity.\
two_or_more,latest.student.demographics.race_ethnicity.non_resident_alien,latest.student.demographics.race_ethnicity.unknown,\
latest.cost.avg_net_price.public,latest.cost.avg_net_price.private,latest.cost.avg_net_price.program_year,latest.cost.avg_net_price.\
other_academic_year,latest.cost.net_price.public.by_income_level.0-30000,latest.cost.net_price.public.by_income_level.30001-48000,\
latest.cost.net_price.public.by_income_level.48001-75000,latest.cost.net_price.public.by_income_level.75001-110000,\
latest.cost.net_price.public.by_income_level.110001-plus,latest.cost.net_price.private.by_income_level.0-30000,latest.cost.net_price.private\
.by_income_level.30001-48000,latest.cost.net_price.private.by_income_level.48001-75000,latest.cost.net_price.private.by_income_level.75001-110000,\
latest.cost.net_price.private.by_income_level.110001-plus,latest.cost.attendance.academic_year,latest.cost.attendance.program_year,\
latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,latest.cost.tuition.program_year,school.faculty_salary,school.ft_faculty_rate,\
latest.aid.federal_loan_rate,latest.student.share_25_older,latest.aid.loan_principal,latest.aid.median_debt.completers.overall,latest.aid.median_debt\
.noncompleters,latest.aid.median_debt.income.0_30000,latest.aid.median_debt.income.30001_75000,latest.aid.median_debt.income.greater_than_75000\
,latest.aid.students_with_any_loan,latest.student.demographics.age_entry,latest.student.demographics.female_share,\
latest.student.demographics.avg_family_income,latest.student.demographics.median_family_income,latest.student.grad_students'


# .at[] avoids FutureWarning
for idx in updatedDF.index:
    school = str(updatedDF.at[idx, 'IPED6'])
    school = ''.join(ch for ch in school if ch.isdigit())
    school = school[:-1]
    response = requests.get(url + 'id=' + school + '&_fields=' + querystring + '&api_key=' + API_key).json()
    if school not in schools.keys():
        schools[school] = response
    else:
        schools[school + 'branch'] = response

In [6]:
updatedDF.head()

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary,OPEID8,Address,City,State,Zip,IPED6,Web,OPEID6
0,Massachusetts Institute of Technology,Engineering,"$72,200.00","$126,000.00","$76,800.00","$99,200.00","$168,000.00","$220,000.00",00217800,77 Massachusetts Avenue,Cambridge,MA,02139-4307,166683.0,web.mit.edu/,2178.0
1,California Institute of Technology,Engineering,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN,00113100,1200 E California Blvd,Pasadena,CA,91125,110404.0,www.caltech.edu,1131.0
2,Harvey Mudd College,Engineering,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN,00117100,301 Platt Blvd.,Claremont,CA,91711,115409.0,www.hmc.edu,1171.0
5,Worcester Polytechnic Institute,Engineering,"$61,000.00","$114,000.00","$80,000.00","$91,200.00","$137,000.00","$180,000.00",00223300,100 Institute Road,Worcester,MA,01609-2280,168421.0,www.wpi.edu,2233.0
6,Carnegie Mellon University,Engineering,"$61,800.00","$111,000.00","$63,300.00","$80,100.00","$150,000.00","$209,000.00",00324200,5000 Forbes Ave,Pittsburgh,PA,15213-3890,211440.0,www.cmu.edu/,3242.0


In [8]:
# this is to just check one result in schools dictionary, note that this API was probably coded
# by some sort of soviet megamind who delights only in the suffering of westerners

#schools['166683']
#print(url + 'id=' + '166683' + '&_fields=' + querystring + '&api_key=' + API_key)

In [7]:
# now that the CSVs are in a DataFrame structure, we can begin using sqlalchemy and pandas to create tables
# actually first I want to put the API data into a DataFrame and relate it with the current dataframes
# save schoolid as int

updatedDF = updatedDF.rename(columns = {'IPED6' : 'schoolid'})
for idx in updatedDF.index:
    updatedDF.at[idx, 'schoolid'] = int(updatedDF.at[idx, 'schoolid'])

# make the schools dictionary a good format for changing into a pandas DataFrame
for school in schools.keys():
    schools[school] = schools[school]['results'][0]

In [29]:
# rename the proto-columns in schools dict
# to avoid sql error with column names being too long
# also changing the schools keys data type to avoid another sql error (it doesn't like blob type data I guess)
# 
# NOTE: this eliminated schools I had designated as branch, after looking through it appeared to be duplicate data
#       but I will keep this note regardless
tempdict = {}
for school in schools.keys():
    if school.isnumeric():
        newschool = int(school)
        tempdict[newschool] = {}
        for procol in (schools[school].keys()):
            temp = schools[school][procol]
            newcol = procol.replace('latest.', '').replace('admissions.', '').replace('academics.', '').replace('program_percentage', 'program%').replace('student.', '').replace('race_ethnicity', 'ethn').replace('cost.', '')
            tempdict[newschool][newcol] = temp


schoolDF = pd.DataFrame.from_dict(tempdict, orient = 'index')
#schoolDF.head()
# index is the IPED6 or 'schoolid' column in updatedDF

In [8]:
############   We currently have 3 active pandas DataFrames to use with MySQL
############   updatedDF contains information about schools vs returns, schoolid relates it to
############   schoolDF which contains all of the API data (costs, academics, demographics)
############   degreeDF contains information about degrees vs returns


# build a mysql connection for use with pd.DF.to_sql() method
from config import rootpw

MySQL_db = 'colleges_db'

engine = create_engine("mysql://root:"+ rootpw +"@localhost/"+MySQL_db)
conn = engine.connect()

updatedDF.to_sql(name = 'school_salary', con = conn, if_exists = 'replace')
schoolDF.to_sql(name = 'school_info', con = conn, if_exists = 'replace', index_label = 'schoolid')
degreeDF.to_sql(name = 'degree_salary', con = conn, if_exists = 'replace')